In [ ]:
import keras.layers
%load_ext autoreload
%autoreload 2
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from watch_recognition.models import build_backbone
from watch_recognition.reports import log_scalar_metrics, run_on_image_debug, generate_report_for_keypoints
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp

from watch_recognition.data_preprocessing import load_keypoints_data_2

from watch_recognition.augmentations import set_shapes, \
    process_kp_data, encode_keypoints_to_mask
from functools import partial

plt.style.use("dark_background")
%matplotlib inline


EPOCHS = 150
image_size = (224, 224)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
        weights="imagenet",  # Load weights pre-trained on ImageNet.
        input_shape=(*image_size, 3),
        include_top=False,
    )
for layer in base_model.layers:
    if 'project_conv' in layer.name:
        print(layer.name, layer.output.shape)

In [ ]:
base_model = build_backbone(image_size)
# base_model.trainable = False

In [ ]:
# build model

inputs = tf.keras.Input(shape=(*image_size, 3), )
x = base_model(inputs)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(2, strides=1,padding='same')(x)
x = tf.keras.layers.Conv2D(
    filters=64, kernel_size=3, strides=1, padding="same", activation='relu'
)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(2, strides=1,padding='same')(x)
output = tf.keras.layers.Conv2D(
    filters=5, kernel_size=1, strides=1, padding="same", activation='softmax'
)(x)


model = tf.keras.models.Model(
    inputs=inputs, outputs=output
)

In [ ]:
model.summary()


In [ ]:
model.output.shape[1:]

In [ ]:


X, y = load_keypoints_data_as_kp(
    Path("../download_data/keypoints/train"),
)
X.shape, y.shape

In [ ]:


dataset = tf.data.Dataset.from_tensor_slices((X, y))
AUTOTUNE = tf.data.experimental.AUTOTUNE


ds_alb = dataset.map(
        process_kp_data,
        num_parallel_calls=AUTOTUNE,
)
ds_alb = ds_alb.map(
        partial(encode_keypoints_to_mask,
                image_size=(224, 224, 3),
                mask_size=(14,14),
                extent=(1,1),
        ),
        num_parallel_calls=AUTOTUNE
)
ds_alb = ds_alb.map(
    partial(set_shapes,
            img_shape=(224, 224, 3),
            target_shape=(14,14, 5)),
    num_parallel_calls=AUTOTUNE).shuffle(8 * 32).batch(32).prefetch(AUTOTUNE)
ds_alb


In [ ]:
from watch_recognition.augmentations import view_image, encode_keypoints_to_mask_np
import numpy as np
view_image(ds_alb)

In [ ]:
X_val, y_val = load_keypoints_data_as_kp(
    Path("../download_data/keypoints/validation"),
)
X_val.shape, y_val.shape

In [ ]:
dataset_val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
dataset_val = dataset_val.map(
        partial(encode_keypoints_to_mask,
                image_size=(224, 224, 3),
                mask_size=(14,14),
                extent=(1,1)),
        num_parallel_calls=AUTOTUNE
)
dataset_val = dataset_val.map(
    partial(set_shapes,
            img_shape=(224, 224, 3),
            target_shape=(14,14, 5)),
    num_parallel_calls=AUTOTUNE).batch(X_val.shape[0]).prefetch(AUTOTUNE)
dataset_val

In [ ]:
view_image(dataset_val)

In [ ]:
# y[0, 0, :2]

In [ ]:
from watch_recognition.reports import visualize_high_loss_examples
from watch_recognition.models import custom_focal_loss

optimizer = tf.keras.optimizers.Adam(3e-4) # use lower lr
# loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-8)
loss = custom_focal_loss
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


start = datetime.now()
TYPE='keypoint'
MODEL_NAME='efficientnetb0'
logdir = f"tensorboard_logs/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}"
print(logdir)
file_writer_distance_metrics_train = tf.summary.create_file_writer(logdir + "/train")
file_writer_distance_metrics_validation = tf.summary.create_file_writer(logdir + "/validation")

model_path = f'models/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}.h5'
model.fit(
    ds_alb,
    epochs=EPOCHS,
    validation_data=dataset_val,
    callbacks=[
        tf.keras.callbacks.TensorBoard(
            log_dir=logdir,
            update_freq="epoch",
        ),
        tf.keras.callbacks.ModelCheckpoint(
                filepath=model_path,
                save_weights_only=False,
                monitor='val_loss',
                save_best_only=True,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', factor=0.8,
                              patience=20, min_lr=1e-6, cooldown=3),
        # tf.keras.callbacks.EarlyStopping(
        #     monitor="val_loss",
        #     restore_best_weights=True,
        #     patience=15,
        # ),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=partial(log_scalar_metrics, X=X, y=y,
                                 file_writer=file_writer_distance_metrics_train, model=model)),
        tf.keras.callbacks.LambdaCallback(
                on_epoch_end=partial(visualize_high_loss_examples,
                                     dataset=ds_alb,
                                     file_writer=file_writer_distance_metrics_train, model=model,
        )),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=partial(log_scalar_metrics, X=X_val, y=y_val,
                                 file_writer=file_writer_distance_metrics_validation, model=model)),


    ]
)
elapsed = (datetime.now() - start).seconds
print(f"total training time: {elapsed / 60} minutes, average: {elapsed/60/EPOCHS} minutes/epoch")

In [ ]:
train_X, train_y = next(iter(ds_alb))
train_X, train_y = train_X.numpy(), train_y.numpy()
run_on_image_debug(model, train_X[0], None, show_grid=False)


In [ ]:
path = Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.14.png")
test_image = tf.keras.preprocessing.image.load_img(
    path, "rgb", target_size=image_size, interpolation="bicubic",
)
test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

In [ ]:
run_on_image_debug(model, test_image_np, show_grid=False)

In [ ]:
generate_report_for_keypoints(model, X, None, show_top_n_errors=5)

In [ ]:
generate_report_for_keypoints(model, X_val, None, show_top_n_errors=5)

In [ ]:
# model_path = "./models/keypoint/efficientnetb0/run_1630519001.517935.h5"
loaded_model = tf.keras.models.load_model(model_path, compile=False)

In [ ]:
for path in [
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.14.png"),
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.24.png"),
    Path("../example_data/test-image-2.jpg"),
]:
    test_image = tf.keras.preprocessing.image.load_img(
        path, "rgb", target_size=image_size, interpolation="bicubic",
    )
    test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

    run_on_image_debug(loaded_model, test_image_np, show_grid=False)

In [ ]:
generate_report_for_keypoints(loaded_model, X, None, show_top_n_errors=5)

In [ ]:
generate_report_for_keypoints(loaded_model, X_val, None, show_top_n_errors=5)

In [ ]:
for image in X_val[:10]:
    run_on_image_debug(loaded_model, image, show_grid=False)

In [ ]:
model_path

In [ ]:
plt.imshow(X[0])

In [ ]:
pred = loaded_model.predict(X[:1])

In [ ]:
for layer in range(3):
    plt.imshow(pred[0, :,:,layer])
    plt.show()

In [ ]:
import numpy as np
from scipy.special import  expit
for layer in range(3):
    output = pred[0, :,:, layer]
    output = np.reshape(output, (14, 14))
    plt.imshow(output)
    plt.show()

In [ ]:
indices = np.argmax(pred[0],axis=-1)
for i in np.unique(indices):
    mask = indices == i
    plt.imshow(mask)
    plt.show()
# TODO add background layer and use softmax to make each output coordinate act like
# multi class classifier

In [ ]:
view_image(ds_alb)

In [ ]:
iterator = ds_alb.take(1).as_numpy_iterator()

In [ ]:
batch = next(iterator)

In [ ]:
X_batch, y_batch = batch

In [ ]:
X_batch.shape, y_batch.shape

In [ ]:
predictions = loaded_model.predict(X_batch)

In [ ]:
loss = tf.keras.metrics.categorical_crossentropy(y_batch, predictions).numpy()

In [ ]:
worst_examples = np.argsort(loss.sum(axis=-1).sum(axis=-1))[:5]

In [ ]:
for X_img in X_batch[worst_examples]:
    plt.imshow(X_img)
    plt.show()

